In [1]:
import neptune
from dotenv import load_dotenv
import os


In [2]:
import json


In [6]:
import neptune

# Initialize Neptune with your workspace and project
load_dotenv()
NEPTUNE_API_KEY = os.getenv("NEPTUNE_API")

project_name = "research-nitrat/SpinsTUM"
# Initialize a connection to the project
project = neptune.init_project(
    project=project_name,
    api_token=NEPTUNE_API_KEY
)

# Fetch all runs as a Pandas DataFrame
runs_df = project.fetch_runs_table().to_pandas()

# Extract run IDs
run_ids = runs_df["sys/id"].tolist()

# Print the run IDs
print("Run IDs:", run_ids)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/research-nitrat/SpinsTUM/


Fetching table...: 0 [00:00, ?/s]

Run IDs: ['SPIN-474', 'SPIN-473', 'SPIN-471', 'SPIN-470', 'SPIN-469', 'SPIN-468', 'SPIN-467', 'SPIN-466', 'SPIN-465', 'SPIN-464', 'SPIN-463', 'SPIN-462', 'SPIN-461', 'SPIN-460', 'SPIN-459', 'SPIN-458', 'SPIN-457', 'SPIN-456', 'SPIN-455', 'SPIN-454', 'SPIN-453', 'SPIN-452', 'SPIN-451', 'SPIN-450', 'SPIN-449', 'SPIN-448', 'SPIN-447', 'SPIN-446', 'SPIN-445', 'SPIN-444', 'SPIN-443', 'SPIN-442', 'SPIN-441', 'SPIN-440', 'SPIN-439', 'SPIN-438', 'SPIN-437', 'SPIN-436', 'SPIN-435', 'SPIN-434', 'SPIN-433', 'SPIN-432', 'SPIN-431', 'SPIN-430', 'SPIN-429', 'SPIN-428', 'SPIN-427', 'SPIN-426', 'SPIN-425', 'SPIN-424', 'SPIN-423', 'SPIN-422', 'SPIN-420', 'SPIN-419', 'SPIN-418', 'SPIN-417', 'SPIN-416', 'SPIN-415', 'SPIN-414', 'SPIN-413', 'SPIN-412', 'SPIN-411', 'SPIN-410', 'SPIN-409', 'SPIN-408', 'SPIN-407', 'SPIN-406', 'SPIN-405', 'SPIN-404', 'SPIN-403', 'SPIN-402', 'SPIN-401', 'SPIN-400', 'SPIN-399', 'SPIN-398', 'SPIN-397', 'SPIN-396', 'SPIN-395', 'SPIN-394', 'SPIN-393', 'SPIN-392', 'SPIN-391', 'SPIN-

In [7]:
run_ids

['SPIN-474',
 'SPIN-473',
 'SPIN-471',
 'SPIN-470',
 'SPIN-469',
 'SPIN-468',
 'SPIN-467',
 'SPIN-466',
 'SPIN-465',
 'SPIN-464',
 'SPIN-463',
 'SPIN-462',
 'SPIN-461',
 'SPIN-460',
 'SPIN-459',
 'SPIN-458',
 'SPIN-457',
 'SPIN-456',
 'SPIN-455',
 'SPIN-454',
 'SPIN-453',
 'SPIN-452',
 'SPIN-451',
 'SPIN-450',
 'SPIN-449',
 'SPIN-448',
 'SPIN-447',
 'SPIN-446',
 'SPIN-445',
 'SPIN-444',
 'SPIN-443',
 'SPIN-442',
 'SPIN-441',
 'SPIN-440',
 'SPIN-439',
 'SPIN-438',
 'SPIN-437',
 'SPIN-436',
 'SPIN-435',
 'SPIN-434',
 'SPIN-433',
 'SPIN-432',
 'SPIN-431',
 'SPIN-430',
 'SPIN-429',
 'SPIN-428',
 'SPIN-427',
 'SPIN-426',
 'SPIN-425',
 'SPIN-424',
 'SPIN-423',
 'SPIN-422',
 'SPIN-420',
 'SPIN-419',
 'SPIN-418',
 'SPIN-417',
 'SPIN-416',
 'SPIN-415',
 'SPIN-414',
 'SPIN-413',
 'SPIN-412',
 'SPIN-411',
 'SPIN-410',
 'SPIN-409',
 'SPIN-408',
 'SPIN-407',
 'SPIN-406',
 'SPIN-405',
 'SPIN-404',
 'SPIN-403',
 'SPIN-402',
 'SPIN-401',
 'SPIN-400',
 'SPIN-399',
 'SPIN-398',
 'SPIN-397',
 'SPIN-396',

In [9]:
import json
from collections import defaultdict

# Initialize Neptune with your workspace and project
load_dotenv()
NEPTUNE_API_KEY = os.getenv("NEPTUNE_API")

project_name = "research-nitrat/SpinsTUM"
api_token = NEPTUNE_API_KEY


# Function to process a single run
def process_run(run_id):
    # Reinitialize the Neptune run
    run = neptune.init_run(
        project=project_name,
        api_token=api_token,
        with_id=run_id
    )

    # Fetch and parse the model summary
    model_summary_json = run["model/summary"].fetch()
    model_summary = json.loads(model_summary_json)

    # Initialize counters and metadata
    architecture_metadata = defaultdict(list)
    num_conv_layers = 0
    num_dense_layers = 0

    for layer in model_summary["config"]["layers"]:
        layer_type = layer["class_name"]
        layer_config = layer["config"]

        # Convolutional layers
        if layer_type == "Conv2D":
            num_conv_layers += 1
            architecture_metadata["conv_layers"].append({
                "filters": layer_config.get("filters"),
                "kernel_size": str(tuple(layer_config.get("kernel_size"))),  # Convert tuple to string
                "activation": layer_config.get("activation"),
                "stride": str(tuple(layer_config.get("strides"))),  # Convert tuple to string
                "padding": layer_config.get("padding")
            })

        # Dense layers
        elif layer_type == "Dense":
            num_dense_layers += 1
            architecture_metadata["dense_layers"].append({
                "units": layer_config.get("units"),
                "activation": layer_config.get("activation")
            })

        # Dropout layers
        elif layer_type == "Dropout":
            architecture_metadata["dropouts"].append({
                "rate": layer_config.get("rate")
            })

    # Aggregate metadata
    architecture_metadata["num_conv_layers"] = num_conv_layers
    architecture_metadata["num_dense_layers"] = num_dense_layers
    architecture_metadata["uses_dropout"] = len(architecture_metadata["dropouts"]) > 0

    # Log architecture metadata
    run["model/num_conv_layers"] = architecture_metadata["num_conv_layers"]
    run["model/num_dense_layers"] = architecture_metadata["num_dense_layers"]
    run["model/uses_dropout"] = architecture_metadata["uses_dropout"]

    # Log detailed layer configurations
    for i, conv_layer in enumerate(architecture_metadata["conv_layers"]):
        for key, value in conv_layer.items():
            run[f"model/conv_layers/{i}/{key}"] = value

    for i, dense_layer in enumerate(architecture_metadata["dense_layers"]):
        for key, value in dense_layer.items():
            run[f"model/dense_layers/{i}/{key}"] = value

    for i, dropout_layer in enumerate(architecture_metadata["dropouts"]):
        run[f"model/dropouts/{i}/rate"] = dropout_layer["rate"]

    # Stop the run
    run.stop()


# Loop through all run IDs and process each one
for run_id in run_ids:
    process_run(run_id)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/research-nitrat/SpinsTUM/e/SPIN-474
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 21 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 21 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/research-nitrat/SpinsTUM/e/SPIN-474/metadata
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/research-nitrat/SpinsTUM/e/SPIN-473
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 18 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 18 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.